**Import packages**

In [6]:
import openai
from openai import OpenAI

from rich.console import Console
from rich.markdown import Markdown
import gradio as gr

import json
import tempfile

**Define OpenAI API Key and Client**

In [ ]:
with open('path/OpenAI_API_Key.txt', 'r') as file:
    api_key = file.read()
openai.api_key = api_key

client = OpenAI(api_key=api_key)

**Define system prompt for Bot character**

In [8]:
def load_dobby_personality():
    with open('Dobby_personality.txt', 'r', encoding='utf-8') as file:
        short_system_prompt = file.read()
    return short_system_prompt

**Create function to chat with Bot**

In [9]:
messages = [
    {"role": "system", "content": load_dobby_personality()},
]


def message_with_chatbot(user_input: str, messages: list[dict] = messages) -> str:
  """Get a response fom a chatbot using OpenAI models.

  Parameters
  ----------
  user_input : str
    Message to send to the chatbot.
  messages : list[dict]
    Conversation history. Tipically contains at least the system message.

  Returns
  -------
  str
    The response of the chatbot to the user question.
  """
  if isinstance(user_input, dict):
    user_input = user_input.get("text")

  messages.append({"role": "user", "content": user_input})

  response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages,
      temperature=0.8,
      max_tokens=256
  )
  response_content = response.choices[0].message.content
  messages.append({"role": "assistant", "content": response_content})
  return response_content

def message_with_dobby(user_input: str):
  short_system_prompt = load_dobby_personality()
  messages = messages = [
    {"role": "system", "content": short_system_prompt}
  ]
  return message_with_chatbot(user_input, messages)

**Create utility function to save data to json file**

In [10]:
def save_json(filepath: str, data: list or dict) -> str:
    """Save data to json file.

    Parameters
    ----------
    filepath : str
      Output filepath.
    data : list | dict
      Data to save to file.

    Returns
    -------
    str
      The confirmation message.
    """
    with open(file=filepath, mode="w", encoding="utf-8") as fp:
        json.dump(obj=data, fp=fp, ensure_ascii=False, indent=4)
    return f"The file {filepath} has been correctly saved"

In [11]:
messages = [
  {"role": "system", "content": load_dobby_personality()},
]

console = Console()

while True:
  user_input = input("Tu: ").strip()
  if user_input.lower() == "q":
    save_json(filepath="dobbybot_chat.json", data=messages)
    break
  dobbybot_response = message_with_dobby(user_input)
  md = Markdown(dobbybot_response)
  console.print(md)

**Create the chatbot with the Gradio interface**

In [ ]:
def save_history(messages: list[list]) -> None:
  """Save chatbot conversation history using Gradio interface.

  Parameters
  ----------
  messages : list[list]
    List of messages saved internally by Gradio.

  Returns
  -------
  None
    The conversation history in json format.
  """
  data = list(map(lambda x: {x[0]: x[1]}, messages))
  save_json(filepath="messages.json", data=data)


def interface(user_input: str, messages: list) -> str:
  """Define chatbot interface for Gradio.

  Parameters
  ----------
  user_input : str
    Message to send to the chatbot.
  messages : list[dict]
    Conversation history. Tipically contains at least the system message.

  Returns
  -------
  str
    The response of the chatbot to the user question.
  """
  chatbot_messages = []
  for message in messages:
    if isinstance(message, str):
      chatbot_messages.append(message)
    else:
      chatbot_messages.append({"role": "user", "content": message[0]})
      chatbot_messages.append({"role": "assistant", "content": message[1]})
  response = message_with_dobby(user_input)
  messages.append((user_input, response))
  save_history(messages)
  return response

messages = [
    {"role": "system", "content": load_dobby_personality()},
]

demo = gr.ChatInterface(
  fn=interface,
  chatbot=gr.Chatbot(height=300),
  textbox=gr.Textbox(placeholder="Ask me something, sir!", container=False, scale=7),
  title="DobbyBot",
  theme="default",
  examples=["Who is your best friend?", "Who are you?", "What kind of abilities do you have?", "What makes you happy?"],
  retry_btn=None,
  undo_btn="Delete Previous",
  clear_btn="Clear"
  )

demo.launch(debug=True)